# Dd's Version

# Classification Project

By Ednalyn C. De Dios & Michael P. Moran

## Project Planning

### Goals
1. Explain what is driving customers to churn

### Deliverables
1. [ ] A report (jupyter notebook) containing analysis of what is driving customer churn
1. [ ] A csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn)
1. [ ] A single google slide that illustrates how your model works, including the features being used
    - Audience: senior leadership team
    - How were the values derived?
    - How likely is the model
        - to give a high probability of churn when churn doesn't occur,
        - to give a low probability of churn when churn occurs, and
        - to accurately predict churn.
1. [ ] A python script that prepares data such that it can be fed into your model
1. [ ] A README.md file that contains a link to your google slides presentation, and instructions for how to use your python script(s)

**Why are our customers churning?**

- Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))
- Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
- Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
- If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

***Deliverables***

1. I will also need a report (ipynb) answering the question, **"Why are our customers churning?"** I want to see the analysis you did to answer my questions and lead to your findings. Please clearly call out the questions and answers you are analyzing. E.g. If you find that month-to-month customers churn more, I won't be surprised, but I am not getting rid of that plan. The fact that they churn is not because they can, it's because they can and they are motivated to do so. I want some insight into why they are motivated to do so. I realize you will not be able to do a full causal experiment, but I hope to see some solid evidence of your conclusions.
1. I will need you to deliver to me a csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn). I would also like a single goolgle slide that illustrates how your model works, including the features being used, so that I can deliver this to the SLT when they come with questions about how these values were derived. Please make sure you include how likely your model is to give a high probability of churn when churn doesn't occur, to give a low probability of churn when churn occurs, and to accurately predict churn.
1. Finally, our development team will need a .py file that will take in a new dataset, (in the exact same form of the one you acquired from telco_churn.customers) and perform all the transformations necessary to run the model you have developed on this new dataset to provide probabilities and predictions.



### Data Dictionary & Domain Knowledge

### Gender

| Description | gender
| ------------- |:-------------:
| Female      | 0
| Male      | 1

### Phone Service

| Phone Service | phone_id
| ------------- |:-------------:
| No phone      | 0
| One Line      | 1
| Two+ lines    | 2

### Household Type

| Partner & Dependents      | household_type_id
| -------------             |:-------------:
| No partner, no dependents      | 0
| Partner, no dependents      | 1
| No partner, dependents    | 2
| Partner, dependents | 3

### Streaming Services

| Streaming TV & Streaming Movies      | streaming_services
| -------------             |:-------------:
| No internet service      | 0
| No streaming tv, no streaming movies      | 1
| Has streaming tv, no streaming movies    | 2
| No streaming tv, has streaming movies | 3
| Has streaming tv, has streaming movies | 4

### Online Security & Backup

| Online Security & Online Backup      | online_security_backup
| -------------             |:-------------:
| No internet service      | 0
| No security, no backup | 1
| Has security, no backup | 2
| No security, has backup | 3
| Has security, has backup | 4


### Hypotheses

### Thoughts & Questions

#### Questions
1. What is SLT?
#### From the boss
1. Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))
1. Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
1. Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
1. If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

## Prepare Environment

In [1]:
from env import host, user, password

import numpy as np
import pandas as pd

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Acquisition

### Grab Data

1. Use the mysql connector to query telco_churn.customers. Assign the output of that query to the dataframe df. You want to include all the fields.

In [2]:
def get_db_url(
    hostname: str, username: str, password: str, db_name: str
) -> str:
    """
    return url for accessing a mysql database
    """
    return f"mysql+pymysql://{username}:{password}@{hostname}/{db_name}"


def get_sql_conn(hostname: str, username: str, password: str, db_name: str):
    """
    return a mysql connection object
    """
    return create_engine(get_db_url(host, user, password, db_name))


def df_from_sql(query: str, url: str) -> pd.DataFrame:
    """
    return a Pandas DataFrame resulting from a sql query
    """
    return pd.read_sql(query, url)


def get_telco_data() -> pd.DataFrame:
    db = "telco_churn"
    query = ("SELECT * "
             f"FROM customers;")
    url = get_db_url(host, user, password, db)
    return df_from_sql(query, url)

In [3]:
df = get_telco_data()

In [4]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.6,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.9,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.9,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.0,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.9,267.4,Yes


### Summarize Data

2. Write a function, peekatdata(dataframe), that takes a dataframe as input and computes and returns the following:

    - creates dataframe object head_df (df of the first 5 rows) and prints contents to screen
    - creates dataframe object tail_df (df of the last 5 rows) and prints contents to screen
    - creates tuple object shape_tuple (tuple of (nrows, ncols)) and prints tuple to screen
    - creates dataframe object describe_df (summary statistics of all numeric variables) and prints contents to screen.
    - prints to screen the information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage.

In [5]:
def peekatdata(dataframe):
    head_df = dataframe.head()
    print(f"HEAD\n{head_df}", end="\n\n")

    tail_df = dataframe.tail()
    print(f"TAIL\n{tail_df}", end="\n\n")

    shape_tuple = dataframe.shape
    print(f"SHAPE: {shape_tuple}", end="\n\n")

    describe_df = dataframe.describe()
    print(f"DESCRIPTION\n{describe_df}", end="\n\n")

    print(f"INFORMATION")
    dataframe.info()

In [6]:
peekatdata(df)

HEAD
  customer_id  gender  senior_citizen partner dependents  tenure  \
0  0002-ORFBO  Female               0     Yes        Yes       9   
1  0003-MKNFE    Male               0      No         No       9   
2  0004-TLHLJ    Male               0      No         No       4   
3  0011-IGKFF    Male               1     Yes         No      13   
4  0013-EXCHZ  Female               1     Yes         No       3   

  phone_service multiple_lines  internet_service_type_id online_security  \
0           Yes             No                         1              No   
1           Yes            Yes                         1              No   
2           Yes             No                         2              No   
3           Yes             No                         2              No   
4           Yes             No                         2              No   

   ...  device_protection tech_support streaming_tv streaming_movies  \
0  ...                 No          Yes          Yes      

## Data Prep

### TODO
- [ ] multiple_lines has "No phone service" for the last row, which is not the same as yes/no that other columns have.
- [ ] convert total_charges column to float. it is an object after reading from sql database

1. Write a function, df_value_counts(dataframe), that takes a dataframe as input and computes and returns the values by frequency for each variable. Use the rule of thumb for your logic on whether or not to use the bins argument. The function will use a for loop and an in statement.

In [7]:
def df_value_counts(dataframe):
    for col in df.columns:
        n = df[col].unique().shape[0]
        col_bins = min(n, 10)
        print(f"{col}:")
        if df[col].dtype in ['int64', 'float64'] and n > 10:
            print(df[col].value_counts(bins=col_bins, sort=False))
        else:
            print(df[col].value_counts())
        print("\n")

In [8]:
df_value_counts(df)

customer_id:
0164-XAIRP    1
8224-DWCKX    1
2378-VTKDH    1
8413-VONUO    1
8590-OHDIW    1
3766-EJLFL    1
9668-PUGNU    1
8111-BKVDS    1
5931-FLJJF    1
6685-XSHHU    1
4318-RAJVY    1
2058-DCJBE    1
5442-BXVND    1
6543-XRMYR    1
2369-UAPKZ    1
2753-JMMCV    1
7296-PIXQY    1
8992-OBVDG    1
3570-YUEKJ    1
0320-DWVTU    1
3745-HRPHI    1
3891-PUQOD    1
2049-BAFNW    1
5294-DMSFH    1
6653-CBBOM    1
2115-BFTIW    1
8008-HAWED    1
4248-QPAVC    1
9680-NIAUV    1
4119-ZYPZY    1
             ..
6848-HJTXY    1
1299-AURJA    1
9378-FXTIZ    1
4878-BUNFV    1
8267-KFGYD    1
3143-JQEGI    1
2692-BUCFV    1
3359-DSRKA    1
9142-XMYJH    1
8495-LJDFO    1
0744-GKNGE    1
7312-XSBAT    1
5797-APWZC    1
6248-BSHKG    1
2794-XIMMO    1
9555-SAHUZ    1
6502-KUGLL    1
6317-YPKDH    1
6358-LYNGM    1
8741-LQOBK    1
3557-HTYWR    1
2320-SLKMB    1
1696-HXOWK    1
4724-WXVWF    1
0568-ONFPC    1
6647-ZEDXT    1
0810-BDHAW    1
3996-ZNWYK    1
8591-TKMZH    1
7295-JOMMD    1
Name: custo

- customer_id has no duplicates
- gender is about even
- customers are mostly not seniors
- about equally split along single/partner
- most customers do not have dependents
- there are many new and many old customers
- overwhelming majority have phone service
- closely split along multiple_lines
- overwhelming majority have internet service
    - more have fiber than DSL
    - most do not have online_security
    - most do not have online_backup
    - most do not have device_protection
    - most do not have tech_support
- billing
    - about evenly split along streaming_tv
    - about evenly splot along streaming_movies
    - most customers are month-to-month
    - most customers are paperless billing
    - most customers pay by some form of check
    - many customers pay less than $30
    - most are pay between 45 and 110 per month
- most have not churned
    - about 1900 have
    

### Handle Missing Values

2. Missing Values:

    - Write a function, that returns a dataframe of the column name and the number of missing values and the percentage of missing values (missing records/total records) for each of the columns that have > 0 missing values.

   - Document your takeaways. For each variable:

        - should you remove the observations with a missing value for that variable?
        - should you remove the variable altogether?
        - is missing equivalent to 0 (or some other constant value) in the specific case of this variable?
        - should you replace the missing values with a value it is most likely to represent (e.g. Are the missing values a result of data integrity issues and should be replaced by the most likely value?)
        - Handle the missing values in the way you recommended above.

In [9]:
def df_missing_vals(dataframe):
    null_count = dataframe.isnull().sum()
    null_percentage = (null_count / dataframe.shape[0]) * 100
    empty_count = pd.Series(((dataframe == " ") | (dataframe == "")).sum())
    return pd.DataFrame({"nmissing": null_count, "percentage": null_percentage, "nempty": empty_count})

# test 
# print(df_missing_vals(pd.DataFrame({"col1": [np.nan, 1, "", np.nan, np.nan], "col2": [2, "", 4, np.nan, 4]})))

In [10]:
print(df_missing_vals(df))

                          nmissing  percentage  nempty
customer_id                      0         0.0       0
gender                           0         0.0       0
senior_citizen                   0         0.0       0
partner                          0         0.0       0
dependents                       0         0.0       0
tenure                           0         0.0       0
phone_service                    0         0.0       0
multiple_lines                   0         0.0       0
internet_service_type_id         0         0.0       0
online_security                  0         0.0       0
online_backup                    0         0.0       0
device_protection                0         0.0       0
tech_support                     0         0.0       0
streaming_tv                     0         0.0       0
streaming_movies                 0         0.0       0
contract_type_id                 0         0.0       0
paperless_billing                0         0.0       0
payment_ty

- Document your takeaways. For each variable:
    - Me: No columns have NaNs
    - Me: total_charges has 11 observations with a " " or "" value
        - [x] let's replace the 11 with their monthly_charges * tenure
        - Dd: the total_charges with " " all had tenure of 0 so they were essentially replaced with zeroes
    - should you remove the observations with a missing value for that variable?
    - should you remove the variable altogether?
    - is missing equivalent to 0 (or some other constant value) in the specific case of this variable?
    - should you replace the missing values with a value it is most likely to represent (e.g. Are the missing values a result of data integrity issues and should be replaced by the most likely value?)

Handle the missing values in the way you recommended above.

In [11]:
# Display total_charges with '' or ' '
df.loc[df['total_charges'] == '']
df.loc[df['total_charges'] == ' ']

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
945,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,1,Yes,...,Yes,Yes,Yes,No,3,No,4,56.05,,No
1731,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,3,No internet service,...,No internet service,No internet service,No internet service,No internet service,3,No,2,20.00,,No
1906,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,1,Yes,...,No,Yes,No,No,3,Yes,3,61.90,,No
2025,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,3,No internet service,...,No internet service,No internet service,No internet service,No internet service,2,Yes,2,19.70,,No
2176,3115-CZMZD,Male,0,No,Yes,0,Yes,No,3,No internet service,...,No internet service,No internet service,No internet service,No internet service,3,No,2,20.25,,No
2250,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,3,No internet service,...,No internet service,No internet service,No internet service,No internet service,3,No,2,25.35,,No
2855,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,1,No,...,Yes,Yes,Yes,No,3,No,2,73.35,,No
3052,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,3,No internet service,...,No internet service,No internet service,No internet service,No internet service,3,No,2,25.75,,No
3118,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,1,Yes,...,Yes,Yes,Yes,No,3,Yes,3,52.55,,No
4054,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,1,Yes,...,Yes,No,Yes,Yes,3,No,2,80.85,,No


In [12]:
df['total_charges'] = np.where(df['total_charges'] == ' ', (df.monthly_charges * df.tenure), df['total_charges'])
print(df_missing_vals(df))
df.head(10)

                          nmissing  percentage  nempty
customer_id                      0         0.0       0
gender                           0         0.0       0
senior_citizen                   0         0.0       0
partner                          0         0.0       0
dependents                       0         0.0       0
tenure                           0         0.0       0
phone_service                    0         0.0       0
multiple_lines                   0         0.0       0
internet_service_type_id         0         0.0       0
online_security                  0         0.0       0
online_backup                    0         0.0       0
device_protection                0         0.0       0
tech_support                     0         0.0       0
streaming_tv                     0         0.0       0
streaming_movies                 0         0.0       0
contract_type_id                 0         0.0       0
paperless_billing                0         0.0       0
payment_ty

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.60,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.90,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.90,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.00,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.90,267.4,Yes
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,No,Yes,Yes,Yes,1,Yes,4,69.40,571.45,No
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,2,Yes,...,Yes,Yes,Yes,Yes,3,Yes,3,109.70,7904.25,No
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,2,Yes,...,No,Yes,No,No,3,Yes,4,84.65,5377.8,No
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,No,No,No,1,Yes,1,48.20,340.35,No
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,Yes,Yes,Yes,3,Yes,2,90.45,5957.9,No


3. Transform churn such that "yes" = 1 and "no" = 0

In [13]:
df['churn'] = df['churn'].map( {'No': 0, 'Yes': 1} ).astype(int)
df.head(10)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.60,593.3,0
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.90,542.4,0
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.90,280.85,1
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.00,1237.85,1
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.90,267.4,1
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,No,Yes,Yes,Yes,1,Yes,4,69.40,571.45,0
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,2,Yes,...,Yes,Yes,Yes,Yes,3,Yes,3,109.70,7904.25,0
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,2,Yes,...,No,Yes,No,No,3,Yes,4,84.65,5377.8,0
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,No,No,No,1,Yes,1,48.20,340.35,0
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,Yes,Yes,Yes,3,Yes,2,90.45,5957.9,0


4. Compute a new feature, tenure_year, that is a result of translating tenure from months to years.

In [14]:
df['tenure_year'] = df.tenure / 12
df.head(10)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,tenure_year
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,Yes,Yes,No,2,Yes,2,65.60,593.3,0,0.750000
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,Yes,1,No,2,59.90,542.4,0,0.750000
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,No,No,No,1,Yes,1,73.90,280.85,1,0.333333
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,No,Yes,Yes,1,Yes,1,98.00,1237.85,1,1.083333
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,Yes,Yes,No,1,Yes,2,83.90,267.4,1,0.250000
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,Yes,Yes,Yes,1,Yes,4,69.40,571.45,0,0.750000
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,2,Yes,...,Yes,Yes,Yes,3,Yes,3,109.70,7904.25,0,5.916667
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,2,Yes,...,Yes,No,No,3,Yes,4,84.65,5377.8,0,5.250000
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,No,No,1,Yes,1,48.20,340.35,0,0.583333
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,Yes,Yes,3,Yes,2,90.45,5957.9,0,5.416667


5. Figure out a way to capture the information contained in phone_service and multiple_lines into a single variable of dtype int. Write a function that will transform the data and place in a new column phone_id in df_sql. Be sure you have documented your function and logic well.

In [15]:
def add_phone_id(df):
    '''
    return DataFrame with a new colum phone_id that combines phone_service and multiple_lines as follows:
    
    # 0 = no phone service
    # 1 = One line
    # 2 = Two+ lines
    
    '''
    df_temp = df.copy()
    df_temp.loc[(df_temp.phone_service == 'No'), 'phone_id'] = int(0)
    df_temp.loc[(df_temp.phone_service == 'Yes') & (df_temp.multiple_lines == 'No'), 'phone_id'] = int(1)
    df_temp.loc[(df_temp.phone_service == 'Yes') & (df_temp.multiple_lines == 'Yes'), 'phone_id'] = int(2)
    df_temp = df_temp.astype({'phone_id': int})
    return df_temp

df_sql = add_phone_id(df)
print(df_sql['phone_id'].dtype)
df_sql.tail(10)
df_sql[['phone_service', 'multiple_lines', 'phone_id']].tail(10)

int64


,phone_service,multiple_lines,phone_id
7033,Yes,No,1
7034,Yes,No,1
7035,Yes,No,1
7036,Yes,No,1
7037,Yes,No,1
7038,Yes,No,1
7039,Yes,Yes,2
7040,Yes,No,1
7041,Yes,No,1
7042,No,No phone service,0


6. Figure out a way to capture the information contained in dependents and partner into a single variable of dtype int. Transform the data and place in a new column household_type_id in df_sql. Be sure you have documented your function and logic well.

In [16]:
def add_household_type_id(df):
    '''
    return DataFrame with a new colum household_type_id that combines partner and dependents as follows:
    
    # 0 = no partner, no dependents
    # 1 = has partner, no dependents
    # 2 = no partner, has dependents
    # 3 = has partner, has dependents
    
    '''
    df_temp = df.copy()
    df_temp.loc[(df_temp.partner == 'No')& (df_temp.dependents == 'No'), 'household_type_id'] = int(0)
    df_temp.loc[(df_temp.partner == 'Yes') & (df_temp.dependents == 'No'), 'household_type_id'] = int(1)
    df_temp.loc[(df_temp.partner == 'No') & (df_temp.dependents == 'Yes'), 'household_type_id'] = int(2)
    df_temp.loc[(df_temp.partner == 'Yes') & (df_temp.dependents == 'Yes'), 'household_type_id'] = int(3)
    df_temp = df_temp.astype({'household_type_id': int})
    return df_temp

df_sql = add_household_type_id(df_sql)
print(df_sql['household_type_id'].dtype)
df_sql.tail(10)
df_sql[['partner', 'dependents', 'household_type_id']].tail(15)

int64


,partner,dependents,household_type_id
7028,Yes,Yes,3
7029,No,No,0
7030,Yes,No,1
7031,No,Yes,2
7032,Yes,Yes,3
7033,No,No,0
7034,Yes,Yes,3
7035,No,No,0
7036,No,No,0
7037,No,No,0


7. Figure out a way to capture the information contained in streaming_tv and streaming_movies into a single variable of dtype int. Transform the data and place in a new column streaming_services in df_sql. Be sure you have documented your function and logic well.

In [17]:

def add_streaming_services(df):
    """
    return DataFrame with a new column streaming_services that combines streaming_tv and streaming_movies columns as follows:
    # 0 = no internet service
    # 1 = no streaming_tv, no streaming_movies
    # 2 = has streaming_tv, no streaming_movies
    # 3 = no streaming_tv, has streaming_movies
    # 4 = has streaming_tv, has streaming_movies
    """
    df_temp = df.copy()
    df_temp.loc[(df_temp.streaming_tv == "No internet service") & (df_temp.streaming_movies == 'No internet service'), "streaming_services"] = int(0)
    df_temp.loc[(df_temp.streaming_tv == "No") & (df_temp.streaming_movies == 'No'), "streaming_services"] = int(1)
    df_temp.loc[(df_temp.streaming_tv == "Yes") & (df_temp.streaming_movies == 'No'), "streaming_services"] = int(2)
    df_temp.loc[(df_temp.streaming_tv == "No") & (df_temp.streaming_movies == 'Yes'), "streaming_services"] = int(3)
    df_temp.loc[(df_temp.streaming_tv == "Yes") & (df_temp.streaming_movies == 'Yes'), "streaming_services"] = int(4)
    df_temp = df_temp.astype({"streaming_services": int})
    
    return df_temp

df_sql = add_streaming_services(df_sql)

8. Figure out a way to capture the information contained in online_security and online_backup into a single variable of dtype int. Transform the data and place in a new column online_security_backup in df_sql. Be sure you have documented your function and logic well.

In [18]:
def add_online_security_backup(df):
    """
    # 0 = no internet service
    # 1 = no online_security, no online_backup
    # 2 = has online_security, no online_backup
    # 3 = no online_security, has online_backup
    # 4 = has online_security, has online_backup
    """
    df_temp = df.copy()
    df_temp.loc[(df_temp.online_security == "No internet service") & (df_temp.online_backup == "No internet service"), "online_security_backup"] = 0
    df_temp.loc[(df_temp.online_security == "No") & (df_temp.online_backup == "No"), "online_security_backup"] = 1
    df_temp.loc[(df_temp.online_security == "Yes") & (df_temp.online_backup == "No"), "online_security_backup"] = 2
    df_temp.loc[(df_temp.online_security == "No") & (df_temp.online_backup == "Yes"), "online_security_backup"] = 3
    df_temp.loc[(df_temp.online_security == "Yes") & (df_temp.online_backup == "Yes"), "online_security_backup"] = 4
    df_temp = df_temp.astype({"online_security_backup": int})
    
    return df_temp

df_sql = add_online_security_backup(df_sql)

9. Data Split

    - Split data into train (70%) & test (30%) samples. You should end with 2 data frames: train_df and test_df

In [19]:
xcols = ['gender', 'senior_citizen', 'internet_service_type_id', 'device_protection', 'tech_support',
         'contract_type_id', 'paperless_billing', 'payment_type_id', 'monthly_charges', 'total_charges',
         'tenure_year', 'phone_id', 'household_type_id', 'streaming_services', 'online_security_backup']
X = df_sql[xcols]
y = df_sql[['churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123, stratify=y)

10. Variable Encoding

    - Write an encoder (fit and transform on train_df) for each non-numeric variable. Use that encoder object to transform on test_df

In [20]:
to_encode = ["gender", "device_protection", "tech_support", "paperless_billing"]
encoders = {}
for col in to_encode:
    encoder = LabelEncoder()
    encoder.fit(X_train[col])
    X_train[col] = encoder.transform(X_train[col])
    X_test[col] = encoder.transform(X_test[col])
    encoders[col] = encoder

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


11. Numeric Scaling

    - Fit a min_max_scaler to train_df. Transform monthly_charges and total_charges variables in train_df using the scaler. Then use the scaler object to transform test_df.

In [21]:
scaler = MinMaxScaler()
scaler.fit(X_train[["monthly_charges", "total_charges"]])
X_train[["monthly_charges", "total_charges"]] = scaler.transform(X_train[["monthly_charges", "total_charges"]])
X_test[["monthly_charges", "total_charges"]] = scaler.transform(X_test[["monthly_charges", "total_charges"]])

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Data Exploration

### Deliverable

*I will also need a report (ipynb) answering the question, "Why are our customers churning?" I want to see the analysis you did to answer my questions and lead to your findings. Please clearly call out the questions and answers you are analyzing. E.g. If you find that **month-to-month customers churn more**, I won't be surprised, but I am not getting rid of that plan. The fact that they churn is not because they can, it's because they can and they are motivated to do so. I want some insight into **why they are motivated to do so**. I realize you will not be able to do a full causal experiment, but I hope to see some solid evidence of your conclusions.*

1. **Could the month in which they signed up influence churn?** i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))

2. **Are there features that indicate a higher propensity to churn?** like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?

3. Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?

4. If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

5. Controlling for services (phone_id, internet_service_type_id, online_security_backup, device_protection, tech_support, and contract_type_id), is the mean monthly_charges of those who have churned significantly different from that of those who have not churned?

6. How much of monthly_charges can be explained by internet_service_type? (hint: correlation test). State your hypotheses and your conclusion clearly.

7. How much of monthly_charges can be explained by internet_service_type + phone service type (0, 1, or multiple lines). State your hypotheses and your conclusion clearly.

8. Create visualizations exploring the interactions of variables (independent with independent and independent with dependent). The goal is to identify features that are related to churn, identify any data integrity issues, understand 'how the data works', e.g. we may find that all who have online services also have device protection. In that case, we don't need both of those. (The visualizations done in your analysis for questions 1-5 count towards the requirements below)

    - Each independent variable (except for customer_id) must be visualized in at least two plots, and at least 1 of those compares the independent variable with the dependent variable.

    - For each plot where x and y are independent variables, add a third dimension (where possible), of churn represented by color.

    - Use subplots when plotting the same type of chart but with different variables.

    - Adjust the axes as necessary to extract information from the visualizations (adjusting the x & y limits, setting the scale where needed, etc.)

    - Add annotations to at least 5 plots with a key takeaway from that plot.

    - Use plots from matplotlib, pandas and seaborn.

    - Use each of the following:

        - sns.heatmap
        - pd.crosstab (with color)
        - pd.scatter_matrix
        - sns.barplot
        - sns.swarmplot
        - sns.pairplot
        - sns.jointplot
        - sns.relplot or plt.scatter
        - sns.distplot or plt.hist
        - sns.boxplot
        - plt.plot
        
    - Use at least one more type of plot that is not included in the list above.

9. What can you say about each variable's relationship to churn, based on your initial exploration? If there appears to be some sort of interaction or correlation, assume there is no causal relationship and brainstorm (and document) ideas on reasons there could be correlation.

    - phone_id
    - internet_service_type_id
    - online_security_backup
    - device_protection
    - tech_support
    - contract_type_id
    - senior_citizen
    - tenure
    - tenure_year
    - monthly_charges
    - total_charges
    - payment_type_id
    - paperless_billing
    - contract_type_id
    - gender

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(train_df.corr(), cmap='Blues', annot=True)

10. Summarize your conclusions, provide clear answers to the specific questions, and summarize any takeaways/action plan from the work above.

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [ ]:
train_df.churn.value_counts().sort_index().plot(kind = "bar",alpha = .5)
plt.title("Distirbution of customers who churned")
plt.grid(b = True , which = "major")

In [ ]:
sns.swarmplot(x="internet_service_type_id", y="monthly_charges", data=train_df, hue="churn", palette="Set2")
ax = sns.boxplot(x="internet_service_type_id", y="monthly_charges", data=train_df,
        showcaps=True,boxprops={'facecolor':'None'},
        showfliers=True,whiskerprops={'linewidth':0})

In [ ]:
pd.crosstab(train_df.internet_service_type_id, train_df.churn, margins=True).style.background_gradient(cmap='YlOrRd')

In [ ]:
from pandas.tools.plotting import scatter_matrix
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter

cmap = cm.get_cmap('gnuplot')
axes = pd.scatter_matrix(
    train_df[['monthly_charges', 'total_charges', 'churn']], marker='o', s=40,
    hist_kwds={'bins':15},  figsize=(9,9), cmap=cmap)
for ax in axes.flatten():
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

In [ ]:
ddtrain_df.fare.describe()

In [ ]:
plt.figure(figsize=(14,8))

p = sns.swarmplot(x='monthly_charges', y='tenure_year', hue='churn', data=train_df)



plt.show()

4. Test the final model (transform, evaluate) on your out-of-sample data (test_df). Summarize the performance. Interpret your results.

In [ ]:
g = sns.PairGrid(train_df)
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)

In [ ]:
cmap = cm.get_cmap('gnuplot')
scatter = pd.scatter_matrix(
    train_df[xcols], marker='o', s=40,
    hist_kwds={'bins':15},  figsize=(9,9), cmap=cmap)

In [ ]:
g = sns.pairplot(train_df)

In [ ]:
g = sns.jointplot(x="tenure_year", y="monthly_charges", data=train_df)

In [ ]:
g = sns.jointplot(x="internet_service_type_id", y="monthly_charges", data=train_df)

In [ ]:
ax = sns.violinplot(x="senior_citizen", y="monthly_charges", hue="churn",
...                     data=train_df, palette="muted", split=True)

In [ ]:
ax = sns.boxplot(x="senior_citizen", y="monthly_charges", hue="churn",
...                     data=train_df, palette="muted")

In [ ]:
ax = sns.swarmplot(x="senior_citizen", y="monthly_charges", hue="churn", data=train_df, palette="muted")


In [ ]:
ax = sns.swarmplot(x="senior_citizen", y="tenure_year", hue="churn", data=train_df, palette="muted")


In [ ]:
ax = sns.barplot(x="senior_citizen", y="monthly_charges", data=train_df)

In [ ]:
g = sns.relplot(x="tenure_year", y="monthly_charges", hue="churn", data=train_df)

In [ ]:
sns.distplot(train_df.monthly_charges, bins=8, kde=False, rug=True);

In [ ]:
g = sns.relplot(x="tenure_year", y="monthly_charges", hue="churn", data=train_df)

## Modeling

1. Feature Selection: Are there any variables that seem to provide limited to no additional information? If so, remove those and assign the new limited dataframe to train_reduced

2. Train (fit, transform, evaluate) a logistic regression model varying your meta-parameters.

In [49]:
# 1. Make the thing
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')
# 2. Fit
logit.fit(X_train, y_train)
# 3. Do stuff
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

# Estimate whether a passenger would survive
y_pred = logit.predict(X_train)

# Estimate the probability of a passenger surviving
y_pred_proba = logit.predict_proba(X_train)

# Model score
print(f'Accuracy of the Logistic Regression classifier on\
        training set (Model Score):\
        {logit.score(X_train, y_train)}')

# Confusion matrix
print(confusion_matrix(y_train, y_pred))

cm = pd.DataFrame(confusion_matrix(y_train, y_pred),
                 columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])

# Classification report
print(classification_report(y_train, y_pred))

Coefficient: 
 [[ 0.00859233  0.32244898 -0.03147942 -0.06605443 -0.33097511 -0.8022377
   0.42068349 -0.13558577  2.30159277  1.88269148 -0.52499292 -0.21374175
  -0.07838231  0.04612379 -0.18882123]]
Intercept: 
 [1.21571204]
Accuracy of the Logistic Regression classifier on        training set (Model Score):        0.7697768762677485
[[2825  797]
 [ 338  970]]
              precision    recall  f1-score   support

           0       0.89      0.78      0.83      3622
           1       0.55      0.74      0.63      1308

   micro avg       0.77      0.77      0.77      4930
   macro avg       0.72      0.76      0.73      4930
weighted avg       0.80      0.77      0.78      4930



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
# 1. Make the thing
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='liblinear')
# 2. Fit
logit.fit(X_train, y_train)
# 3. Do stuff
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

# Estimate whether a passenger would survive
y_pred = logit.predict(X_train)

# Estimate the probability of a passenger surviving
y_pred_proba = logit.predict_proba(X_train)

# Model score
print(f'Accuracy of the Logistic Regression classifier on\
        training set (Model Score):\
        {logit.score(X_train, y_train)}')

# Confusion matrix
print(confusion_matrix(y_train, y_pred))

cm = pd.DataFrame(confusion_matrix(y_train, y_pred),
                 columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])

# Classification report
print(classification_report(y_train, y_pred))

Coefficient: 
 [[ 0.01142346  0.32401866 -0.02259342 -0.06613591 -0.33066211 -0.79704295
   0.42423004 -0.1338495   2.30193055  1.84028295 -0.52238604 -0.21087297
  -0.07797986  0.04859171 -0.18572189]]
Intercept: 
 [1.17339325]
Accuracy of the Logistic Regression classifier on        training set (Model Score):        0.7695740365111562
[[2825  797]
 [ 339  969]]
              precision    recall  f1-score   support

           0       0.89      0.78      0.83      3622
           1       0.55      0.74      0.63      1308

   micro avg       0.77      0.77      0.77      4930
   macro avg       0.72      0.76      0.73      4930
weighted avg       0.80      0.77      0.78      4930



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
# 1. Make the thing
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='lbfgs')
# 2. Fit
logit.fit(X_train, y_train)
# 3. Do stuff
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

# Estimate whether a passenger would survive
y_pred = logit.predict(X_train)

# Estimate the probability of a passenger surviving
y_pred_proba = logit.predict_proba(X_train)

# Model score
print(f'Accuracy of the Logistic Regression classifier on\
        training set (Model Score):\
        {logit.score(X_train, y_train)}')

# Confusion matrix
print(confusion_matrix(y_train, y_pred))

cm = pd.DataFrame(confusion_matrix(y_train, y_pred),
                 columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])

# Classification report
print(classification_report(y_train, y_pred))

Coefficient: 
 [[ 0.0084607   0.322414   -0.03148439 -0.06610915 -0.33095022 -0.80225761
   0.4205275  -0.13555418  2.3011728   1.88346574 -0.52511856 -0.21372307
  -0.07841349  0.04615159 -0.18879774]]
Intercept: 
 [1.21615896]
Accuracy of the Logistic Regression classifier on        training set (Model Score):        0.7697768762677485
[[2825  797]
 [ 338  970]]
              precision    recall  f1-score   support

           0       0.89      0.78      0.83      3622
           1       0.55      0.74      0.63      1308

   micro avg       0.77      0.77      0.77      4930
   macro avg       0.72      0.76      0.73      4930
weighted avg       0.80      0.77      0.78      4930



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3. Compare evaluation metrics across all the models, and select the best performing model.

# Delivery

1. I will need you to deliver to me a csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn). I would also like a single google slide that illustrates how your model works, including the features being used, so that I can deliver this to the SLT when they come with questions about how these values were derived. Please make sure you include how likely your model is to give a high probability of churn when churn doesn't occur, to give a low probability of churn when churn occurs, and to accurately predict churn.

1. Finally, our development team will need a .py file that will take in a new dataset, (in the exact same form of the one you acquired from telco_churn.customers) and perform all the transformations necessary to run the model you have developed on this new dataset to provide probabilities and predictions.